In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [2]:
add_train = pd.read_csv('./n_train.csv')
add_test = pd.read_csv('./n_test.csv')

In [3]:
climate_type_dict = {'1':'BSk',
 '9': 'Dfb',
 '4': 'Cfa',
 '7': 'Csb',
 '8': 'Dfa',
 '3': 'BWk',
 '10': 'Dfc',
 '2': 'BWh',
 '6': 'Csa',
 '11': 'Dsb',
 '0': 'BSh',
 '5': 'Cfb',
 '12': 'Dsc',
 '13': 'Dwa',
 '14': 'Dwb'}

In [4]:
all_cols = add_train.columns.tolist()

In [5]:
target=["contest-tmp2m-14d__tmp2m"]

In [6]:
train_df =  pd.read_csv('fe_v1_train.csv')
test_df  =  pd.read_csv('fe_v1_test.csv')
for c in train_df.columns:
    if train_df[c].dtypes != 'float64':
        print(c)

climateregions__climateregion


In [7]:
exclude_cols = ['index', 'startdate']
best_cols = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(best_cols))

Main features: 63


In [8]:
for c in all_cols:
    if 'contest-precip-14d' in c:
        print(c)

contest-precip-14d__precip


In [9]:
def calculate_season(month):
    if month == 12 or month == 1 or month == 2:
        return 4
    if month == 9 or month == 10 or month == 11:
        return 3
    if month == 7 or month == 8 or month == 6:
        return 2
    if month == 3 or month == 4 or month == 5:
        return 1

In [10]:
train_df['month'] = add_train['month']
test_df['month'] = add_test['month']

train_df['season'] = train_df['month'].apply(calculate_season)
test_df['season'] = test_df['month'].apply(calculate_season)

train_df['loc_group'] = add_train['loc_group']
test_df['loc_group'] = add_test['loc_group']

train_df['year'] = add_train['year']
test_df['year'] = add_test['year']

def map_climate_type(x):
    return climate_type_dict[str(x)]

train_df['climate_type'] = train_df['climateregions__climateregion'].apply(map_climate_type)
test_df['climate_type'] = train_df['climateregions__climateregion'].apply(map_climate_type)

precip_col = 'contest-precip-14d__precip'
wind_col = 'contest-wind-h500-14d__wind-hgt-500'

group_cols = ['loc_group', 'climate_type', 'year', 'season', 'month']
season_group_cols = ['loc_group', 'climate_type', 'year', 'season']


In [11]:
slp_col = 'contest-slp-14d__slp'
ccsm3_col = 'nmme-tmp2m-34w__ccsm3'
pres_col = 'contest-pres-sfc-gauss-14d__pres'
precip_col = 'contest-precip-14d__precip'
stt_col = 'sst-2010-1'
sst_col_5 = 'sst-2010-5'
sst_col_3 = 'sst-2010-3'

In [12]:
print(train_df[pres_col].min(), train_df[pres_col].max())
print(test_df[pres_col].min(), test_df[pres_col].max())

74839.94 102845.57
75925.02 102119.93


In [13]:
print(train_df[stt_col].min(), train_df[stt_col].max())
print(test_df[stt_col].min(), test_df[stt_col].max())

-219.87 352.2
89.73 345.33


In [14]:
print(train_df[sst_col_5].min(), train_df[sst_col_5].max())
print(test_df[sst_col_5].min(), test_df[sst_col_5].max())

-32.81 40.97
-4.62 21.5


In [15]:
print(train_df[precip_col].min(), train_df[precip_col].max())
print(test_df[precip_col].min(), test_df[precip_col].max())

0.0 502.63
0.0 439.54


In [16]:
# train_df[pres_col] = train_df[pres_col] - 100000
# test_df[pres_col] = test_df[pres_col] - 100000

In [17]:
print(train_df[slp_col].min(), train_df[slp_col].max())
print(test_df[slp_col].min(), test_df[slp_col].max())

100379.21 103275.84
100593.22 102258.27


In [18]:
print(train_df[wind_col].min(), train_df[wind_col].max())
print(test_df[wind_col].min(), test_df[wind_col].max())

5289.25 5958.2
5504.59 5925.56


In [19]:
train_df[wind_col] = train_df[wind_col] - 5000
test_df[wind_col] = test_df[wind_col] - 5000

In [20]:
train_df[slp_col] = train_df[slp_col] - 100000
test_df[slp_col] = test_df[slp_col] - 100000

In [21]:
check_cols = [c for c in train_df.columns if c not in target and c not in exclude_cols]

df = pd.concat([train_df[check_cols], test_df[check_cols]], axis=0)
df.shape

(407088, 68)

In [22]:
def create_df_by_group_and_agg(dataframe, grouped_cols, action, my_col, return_name):
    if action == "first":
        ans_df = dataframe.groupby(grouped_cols)[my_col].first().reset_index()
    elif action == "min":
        ans_df = dataframe.groupby(grouped_cols)[my_col].min().reset_index()
    elif action == "max":
        ans_df = dataframe.groupby(grouped_cols)[my_col].max().reset_index()
    elif action == "mean":
        ans_df = dataframe.groupby(grouped_cols)[my_col].mean().reset_index()
    print("Nan")
    ans_df = ans_df.rename(columns={my_col: return_name})
    for c in ans_df.columns:
        if ans_df[c].isnull().sum() > 0:
            print(c, ans_df[c].isnull().sum())
    return ans_df

In [23]:
def assign_train_test_df(my_col):
    train_df[my_col] = df[:len(train_df)][my_col].tolist()
    test_df[my_col] = df[len(train_df):][my_col].tolist()

In [24]:
first_slp_df = create_df_by_group_and_agg(df, season_group_cols, "first", slp_col, "first_slp")
first_slp_df.head(3)

Nan


,loc_group,climate_type,year,season,first_slp
0,0,BSh,2014,3,1352.08
1,0,BSh,2014,4,2072.81
2,0,BSh,2015,1,1917.24


In [25]:
first_wind_df = create_df_by_group_and_agg(df, season_group_cols, "first", wind_col, "first_wind")
first_wind_df.head(3)

Nan


,loc_group,climate_type,year,season,first_wind
0,0,BSh,2014,3,899.66
1,0,BSh,2014,4,847.35
2,0,BSh,2015,1,804.47


In [26]:
first_precip_df = create_df_by_group_and_agg(df, season_group_cols, "first", precip_col, "first_precip")
first_precip_df.head(3)

Nan


,loc_group,climate_type,year,season,first_precip
0,0,BSh,2014,3,94.31
1,0,BSh,2014,4,25.53
2,0,BSh,2015,1,31.94


In [27]:
first_ccsm3_df = create_df_by_group_and_agg(df, season_group_cols, "first", ccsm3_col, "first_ccsm3")
first_ccsm3_df.head(3)

Nan


,loc_group,climate_type,year,season,first_ccsm3
0,0,BSh,2014,3,27.57
1,0,BSh,2014,4,6.66
2,0,BSh,2015,1,15.25


In [28]:
first_pres_df = create_df_by_group_and_agg(df, season_group_cols, "first", ccsm3_col, "first_pres")
first_pres_df.head(3)

Nan


,loc_group,climate_type,year,season,first_pres
0,0,BSh,2014,3,27.57
1,0,BSh,2014,4,6.66
2,0,BSh,2015,1,15.25


In [29]:
first_sst_df = create_df_by_group_and_agg(df, season_group_cols, "first", stt_col, "first_sst")
first_sst_df.head(3)

Nan


,loc_group,climate_type,year,season,first_sst
0,0,BSh,2014,3,352.20
1,0,BSh,2014,4,-27.73
2,0,BSh,2015,1,-182.10


In [30]:
first_sst_5_df = create_df_by_group_and_agg(df, season_group_cols, "first", sst_col_5, "first_sst_5")
first_sst_5_df.head(3)

Nan


,loc_group,climate_type,year,season,first_sst_5
0,0,BSh,2014,3,19.29
1,0,BSh,2014,4,24.82
2,0,BSh,2015,1,23.47


In [31]:
first_sst_3_df = create_df_by_group_and_agg(df, season_group_cols, "first", sst_col_3, "first_sst_3")
first_sst_3_df.head(3)

Nan


,loc_group,climate_type,year,season,first_sst_3
0,0,BSh,2014,3,-19.69
1,0,BSh,2014,4,-49.72
2,0,BSh,2015,1,-52.32


In [32]:
min_sst_df = create_df_by_group_and_agg(df, season_group_cols, "min", stt_col, "min_sst")
min_sst_df.head(3)

Nan


,loc_group,climate_type,year,season,min_sst
0,0,BSh,2014,3,-24.17
1,0,BSh,2014,4,-110.84
2,0,BSh,2015,1,-183.70


In [33]:
min_sst_5_df = create_df_by_group_and_agg(df, season_group_cols, "min", sst_col_5, "min_sst_5")
min_sst_5_df.head(3)

Nan


,loc_group,climate_type,year,season,min_sst_5
0,0,BSh,2014,3,7.98
1,0,BSh,2014,4,24.81
2,0,BSh,2015,1,-7.76


In [34]:
min_sst_3_df = create_df_by_group_and_agg(df, season_group_cols, "min", sst_col_3, "min_sst_3")
min_sst_3_df.head(3)

Nan


,loc_group,climate_type,year,season,min_sst_3
0,0,BSh,2014,3,-51.03
1,0,BSh,2014,4,-49.72
2,0,BSh,2015,1,-54.28


In [35]:
max_sst_df = create_df_by_group_and_agg(df, season_group_cols, "max", stt_col, "max_sst")
max_sst_df.head(3)

Nan


,loc_group,climate_type,year,season,max_sst
0,0,BSh,2014,3,352.20
1,0,BSh,2014,4,-27.73
2,0,BSh,2015,1,15.72


In [36]:
max_sst_5_df = create_df_by_group_and_agg(df, season_group_cols, "max", sst_col_5, "max_sst_5")
max_sst_5_df.head(3)

Nan


,loc_group,climate_type,year,season,max_sst_5
0,0,BSh,2014,3,24.50
1,0,BSh,2014,4,27.65
2,0,BSh,2015,1,23.47


In [37]:
max_sst_3_df = create_df_by_group_and_agg(df, season_group_cols, "max", sst_col_3, "max_sst_3")
max_sst_3_df.head(3)

Nan


,loc_group,climate_type,year,season,max_sst_3
0,0,BSh,2014,3,-19.69
1,0,BSh,2014,4,-31.16
2,0,BSh,2015,1,-15.60


In [38]:
first_slp_df_month = create_df_by_group_and_agg(df, group_cols, "first", slp_col, "first_slp_month")
first_slp_df_month.head(3)

Nan


,loc_group,climate_type,year,season,month,first_slp_month
0,0,BSh,2014,3,9,1352.08
1,0,BSh,2014,3,10,1254.96
2,0,BSh,2014,3,11,1952.19


In [39]:
df = df.merge(first_slp_df, how='left', on=season_group_cols)
df['diff_slp_first'] = df[slp_col] - df['first_slp']
assign_train_test_df('diff_slp_first')

In [40]:
df = df.merge(first_precip_df, how='left', on=season_group_cols)
df['diff_precip_first'] = df[precip_col] - df['first_precip']
assign_train_test_df('diff_precip_first')

In [41]:
df = df.merge(first_sst_df, how='left', on=season_group_cols)
df['diff_sst_first'] = df[stt_col] - df['first_sst']
assign_train_test_df('diff_sst_first')

In [42]:
df = df.merge(first_sst_5_df, how='left', on=season_group_cols)
df['diff_sst_5_first'] = df[sst_col_5] - df['first_sst_5']
assign_train_test_df('diff_sst_5_first')

In [43]:
df = df.merge(first_sst_3_df, how='left', on=season_group_cols)
df['diff_sst_3_first'] = df[sst_col_3] - df['first_sst_3']
assign_train_test_df('diff_sst_3_first')

In [44]:
df = df.merge(min_sst_df, how='left', on=season_group_cols)
df['diff_sst_min'] = df[stt_col] - df['min_sst']
assign_train_test_df('diff_sst_min')

In [45]:
df = df.merge(max_sst_df, how='left', on=season_group_cols)
df['diff_sst_max'] = df['max_sst'] - df[stt_col]
assign_train_test_df('diff_sst_max')

In [46]:
df = df.merge(max_sst_5_df, how='left', on=season_group_cols)
df['diff_sst_5_max'] = df[sst_col_5] - df['max_sst_5']
assign_train_test_df('diff_sst_5_max')

In [47]:
df = df.merge(max_sst_3_df, how='left', on=season_group_cols)
df['diff_sst_3_max'] = df[sst_col_3] - df['max_sst_3']
assign_train_test_df('diff_sst_3_max')

In [48]:
df = df.merge(min_sst_5_df, how='left', on=season_group_cols)
df['diff_sst_5_min'] = df[sst_col_5] - df['min_sst_5']
assign_train_test_df('diff_sst_5_min')

In [49]:
df = df.merge(min_sst_3_df, how='left', on=season_group_cols)
df['diff_sst_3_min'] = df[sst_col_3] - df['min_sst_3']
assign_train_test_df('diff_sst_3_min')

In [50]:
df = df.merge(first_pres_df, how='left', on=season_group_cols)
df['diff_pres_first'] = df[pres_col] - df['first_pres']
assign_train_test_df('diff_pres_first')

In [51]:
df = df.merge(first_wind_df, how='left', on=season_group_cols)
df['diff_wind_first'] = df[wind_col] - df['first_wind']
assign_train_test_df('diff_wind_first')

In [52]:
df = df.merge(first_ccsm3_df, how='left', on=season_group_cols)
df['diff_ccsm3_first'] = df[ccsm3_col] - df['first_ccsm3']
assign_train_test_df('diff_ccsm3_first')

In [53]:
# df[f'{precip_col}_bins'] = pd.qcut(df[precip_col], q=5, labels=[f'{precip_col}_{i}' for i in range(5)])
# le = LabelEncoder().fit(np.unique(df[f'{precip_col}_bins'].unique().tolist()))
# df[f'{precip_col}_bins'] = le.transform(df[f'{precip_col}_bins']) + 1
# df[f'{precip_col}_bins'] = df[f'{precip_col}_bins'].replace(np.nan, 0).astype('int').astype('category')
# assign_train_test_df(f'{precip_col}_bins')

In [54]:
df['range_sst'] = df['max_sst'] - df['min_sst']
assign_train_test_df('range_sst')

In [55]:
df['range_sst_5'] = df['max_sst_5'] - df['min_sst_5']
assign_train_test_df('range_sst_5')

In [56]:
df['range_sst_3'] = df['max_sst_3'] - df['min_sst_3']
assign_train_test_df('range_sst_3')

In [57]:
features = best_cols + [
    'diff_slp_first',
    'diff_wind_first',
    'diff_sst_first',
    'diff_precip_first',
    'diff_sst_min',
    'diff_sst_max',
    'range_sst',
    'diff_sst_5_first',
    'diff_sst_5_min',
    'diff_sst_5_max',
    'range_sst_5',
    'diff_sst_3_first',
    'diff_sst_3_min',
    'diff_sst_3_max',
    'range_sst_3'
]

In [58]:
X = train_df[features].values
y = train_df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])    
def train_catboost(x, y):
    print(x.shape, y.shape)
    
    cat_models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = CatBoostRegressor(iterations=5000, verbose=200, task_type="GPU",
                           devices='0:1')
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=200, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models
cat_models = train_catboost(X, y)

preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(test_df[features].values))
s = 0
for pred in preds:
    s += pred
s = s/5
res_df = pd.read_csv('./sample_solution.csv')
res_df['contest-tmp2m-14d__tmp2m'] = s
res_df.to_csv('submission_fe_v2.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

5it [00:04,  1.06it/s]


(375734, 78) (375734,)


  0%|                                                                                                | 0/5 [00:00<?, ?it/s]

Learning rate set to 0.072699
0:	learn: 9.2611105	test: 9.2407165	best: 9.2407165 (0)	total: 6.07ms	remaining: 30.3s
200:	learn: 1.1821373	test: 1.1875026	best: 1.1875026 (200)	total: 1.09s	remaining: 26.1s
400:	learn: 0.9183588	test: 0.9259505	best: 0.9259505 (400)	total: 2.14s	remaining: 24.6s
600:	learn: 0.7919272	test: 0.8020935	best: 0.8020935 (600)	total: 3.1s	remaining: 22.7s
800:	learn: 0.7075547	test: 0.7192735	best: 0.7192735 (800)	total: 4.08s	remaining: 21.4s
1000:	learn: 0.6491738	test: 0.6625967	best: 0.6625967 (1000)	total: 5.03s	remaining: 20.1s
1200:	learn: 0.6034376	test: 0.6182350	best: 0.6182350 (1200)	total: 6.06s	remaining: 19.2s
1400:	learn: 0.5672383	test: 0.5836156	best: 0.5836156 (1400)	total: 7.07s	remaining: 18.2s
1600:	learn: 0.5372627	test: 0.5548642	best: 0.5548642 (1600)	total: 8.12s	remaining: 17.2s
1800:	learn: 0.5117500	test: 0.5303159	best: 0.5303159 (1800)	total: 9.11s	remaining: 16.2s
2000:	learn: 0.4893700	test: 0.5090594	best: 0.5090594 (2000)	to

 20%|█████████████████▌                                                                      | 1/5 [00:36<02:27, 36.89s/it]

...

Learning rate set to 0.072699
0:	learn: 9.2604847	test: 9.2566518	best: 9.2566518 (0)	total: 6.75ms	remaining: 33.7s
200:	learn: 1.1723886	test: 1.1873714	best: 1.1873714 (200)	total: 914ms	remaining: 21.8s
400:	learn: 0.9139956	test: 0.9277570	best: 0.9277570 (400)	total: 1.86s	remaining: 21.3s
600:	learn: 0.7865061	test: 0.8002421	best: 0.8002421 (600)	total: 2.76s	remaining: 20.2s
800:	learn: 0.7009407	test: 0.7145692	best: 0.7145692 (800)	total: 3.68s	remaining: 19.3s
1000:	learn: 0.6433385	test: 0.6582231	best: 0.6582231 (1000)	total: 4.62s	remaining: 18.5s
1200:	learn: 0.5977018	test: 0.6134905	best: 0.6134905 (1200)	total: 5.61s	remaining: 17.7s
1400:	learn: 0.5623695	test: 0.5790099	best: 0.5790099 (1400)	total: 6.56s	remaining: 16.8s
1600:	learn: 0.5324388	test: 0.5499149	best: 0.5499149 (1600)	total: 7.44s	remaining: 15.8s
1800:	learn: 0.5074598	test: 0.5256680	best: 0.5256680 (1800)	total: 8.41s	remaining: 14.9s
2000:	learn: 0.4864087	test: 0.5056668	best: 0.5056668 (2000)	t

 80%|██████████████████████████████████████████████████████████████████████▍                 | 4/5 [02:21<00:35, 35.14s/it]

Learning rate set to 0.072699
0:	learn: 9.2625918	test: 9.2370575	best: 9.2370575 (0)	total: 6.56ms	remaining: 32.8s
200:	learn: 1.1735731	test: 1.1787869	best: 1.1787869 (200)	total: 1.03s	remaining: 24.6s
400:	learn: 0.9144807	test: 0.9225820	best: 0.9225820 (400)	total: 1.97s	remaining: 22.6s
600:	learn: 0.7829890	test: 0.7924937	best: 0.7924937 (600)	total: 3.03s	remaining: 22.2s
800:	learn: 0.7025451	test: 0.7140124	best: 0.7140124 (800)	total: 3.92s	remaining: 20.5s
1000:	learn: 0.6416625	test: 0.6544759	best: 0.6544759 (1000)	total: 4.85s	remaining: 19.4s
1200:	learn: 0.5990840	test: 0.6135165	best: 0.6135165 (1200)	total: 5.8s	remaining: 18.3s
1400:	learn: 0.5624976	test: 0.5780879	best: 0.5780879 (1400)	total: 6.83s	remaining: 17.5s
1600:	learn: 0.5336735	test: 0.5503545	best: 0.5503545 (1600)	total: 7.86s	remaining: 16.7s
1800:	learn: 0.5089837	test: 0.5266715	best: 0.5266715 (1800)	total: 8.82s	remaining: 15.7s
2000:	learn: 0.4871649	test: 0.5057357	best: 0.5057357 (2000)	to

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 26.01it/s]


cos_sim with best submission: 0.999790474634046
